### Goal: to do residual prediction learning:
1. I can do that as the DQN implementation as OpenAI baselines
    - I can try to create another network with shared variables but stop gradients
2. But I want to do it in a simpler way: just take out the current constant value and plug it back!
    - 2.1 I can use placeholder to input it from the outside

In [2]:
import tensorflow as tf
import numpy as np

/is/sg/klee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [31]:
# try a simple linear model
tf.reset_default_graph()
x = tf.placeholder(shape=[None, 3], dtype=tf.float32)

y = model_1(x, 'model_1')
y_2 = model_1(x, 'model_2', reuse=True)

init = tf.global_variables_initializer()

In [15]:
def model_1(x, name_scope, reuse=False):
    with tf.variable_scope(name_scope, reuse=reuse):    
        #np.random.seed(10)
        c = np.random.normal(size=[3,2], ).astype(np.float32)
        W = tf.Variable(initial_value=c)
        y = tf.matmul(x, W)
    return y

In [28]:
feed_dict = {x: [[1,2,3]]}

sess = tf.InteractiveSession()
sess.run(init)
y_ = y.eval(feed_dict=feed_dict) # I wanted to get this value in fixed
y_2_ = y_2.eval(feed_dict=feed_dict)
print('y = {}; y_2 = {}'.format(y_, y_2_))
sess.close()

y = [[ 0.10479391 -1.4617453 ]]; y_2 = [[ 0.10479391 -1.4617453 ]]


In [4]:
def model_2(name_scope, reuse=False):
    with tf.variable_scope(name_scope, reuse=reuse):
        c = np.random.normal()
        s = tf.Variable(c)
        op = tf.assign_add(s,2)
    return s, op

In [37]:
def model_2():    
    #c = np.random.normal()
    s = tf.Variable(1)
    op = tf.assign_add(s,2)
    return s, op

In [38]:
tf.reset_default_graph()
name_scope = 's'
with tf.variable_scope(name_scope):
    s, op = model_2()

with tf.variable_scope(name_scope, reuse=True):
    s_1, _ = model_2()


In [44]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)
print('before: {}'.format(s.eval()))
print('before_1: {}'.format(s_1.eval()))
sess.run(op)
print('after: {}'.format(s.eval()))
print('after_1: {}'.format(s_1.eval()))

before: 1
before_1: 1
after: 3
after_1: 1


In [45]:
tf.global_variables()

[<tf.Variable 's/Variable:0' shape=() dtype=int32_ref>,
 <tf.Variable 's_1/Variable:0' shape=() dtype=int32_ref>]